In [2]:
import os
import re
""" 創建一個用於詞法分析的標記token """
class Token:
    # 設定它的標記種類(type)跟標記值(value)以及標記位置(pos)
    # pos的位置式可選擇的
    def __init__(self, type, value, pos=None):
        # 將傳入的類型和數值賦予token
        self.type = type
        self.value = value
# 代数运算符
PLUS    = 'PLUS'      # 加號
MINUS   = 'MINUS'     # 減號
MUL     = 'MUL'       # 乘號
DIVIDE  = 'DIVIDE'    # 除號
# 作用域符号
LPAREN  = 'LPAREN'    # 左括號
RPAREN  = 'RPAREN'    # 右括號
BEGIN   = 'BEGIN'     # 開始
END     = 'END'       # 结束

# 符号
ID      = 'ID'        # 標示符
SEMI    = 'SEMI'      # 分號
EOF     = 'EOF'       # 文件结束標誌

# 保留关键字
INT     = 'INT'       # 整数類型
RETURN  = 'RETURN'    # 返回關键字
CHAR    = 'CHAR'  
# 頭文件
TT_INCLUDE = 'TT_INCLUDE'
TT_HASH = 'TT_HASH'   # 井字符號
TT_FILENAME = 'TT_FILENAME'

In [3]:

""" 創建一個詞法分析器Lexer"""
class Lexer:
    def __init__(self, text):
        self.text = text  # 將輸入的字串賦予text
        self.pos = 0      # 將最開始的字串位置設定為0
        self.current_char = self.text[self.pos] # 將current_char設定為text目前的pos位置
        self.line = 1
        # Ex: int a 
        # self.text = "int a"
        # self.pos = 0
        # self.current_char = text[o] = "i"

    """移动pos指针到下一个位置。"""
    def next(self):
        self.pos += 1
        if self.pos > len(self.text) - 1: 
            self.current_char = None  # 表示文本輸入結束
        else:
            self.current_char = self.text[self.pos]
        if self.current_char == '\n':
            self.line += 1
    """下一個字符"""
    def get_next_token(self):
        while self.current_char is not None:
            # 如果當前字符是空白鍵則呼叫skip_whitespace()函數
            if self.current_char.isspace():
                self.skip_whitespace()
                continue

            # 如果當前字符是英文則呼叫identifier()函數
            if self.current_char.isalpha():
                if self.modules():
                    continue
                if self.macros():
                    continue
                return self.identifier()
            
            # 如果當前字符是數字則呼叫number()函數
            if self.current_char.isdigit():
                return self.number()

            """
            如果當前字符是`+`,`-`,`*`,`(`,`)`,`{`,`}`,`;`
            則呼叫Token將它替換
            """
            if self.current_char == '+':    
                self.next()
                return Token(PLUS, '+')

            if self.current_char == '-':
                self.next()
                return Token(MINUS, '-')

            if self.current_char == '*':
                self.next()
                return Token(MUL, '*')
            
            if self.current_char == '/':
                self.next()
                return Token(DIVIDE, '/')

            if self.current_char == '(':
                self.next()
                return Token(LPAREN, '(')

            if self.current_char == ')':
                self.next()
                return Token(RPAREN, ')')

            if self.current_char == '{':
                self.next()
                return Token(BEGIN, '{')

            if self.current_char == '}':
                self.next()
                return Token(END, '}')

            if self.current_char == ';':
                self.next()
                return Token(SEMI, ';')
            self.next()
        # 如果current_char = None 則回傳EOF
        return Token(EOF, None)
    
    """處理當前字符為數字型態的函數"""
    def number(self):
        result = ''
        while self.current_char is not None \
                and self.current_char.isdigit():
            result += self.current_char
            self.next()

        return Token(INT, int(result), self.line) # self.line用來回傳目前的行數
    
    """處理當前字符為空白建的函數"""
    def skip_whitespace(self):
        # isspace contains '', '\n', '\t', etc.
        while self.current_char is not None \
                and self.current_char.isspace():

            self.next()

    """處理當前字符為英文型態的函數"""
    def identifier(self):
        result = ''
        # 確認當前字母是否為英文、數字或底線
        while self.current_char is not None and (self.current_char.isalnum() or self.current_char == '_'):
            result += self.current_char
            self.next()
                                                                         # result.upper() 將result結果轉換成大寫，以助於更能正確的匹配關鍵字
        token = RESERVED_KEYWORDS.get(result.upper(), Token(ID, result)) # Token(ID, result) 這段是當R ESERVED_KEYWORDS.get 沒有找到對應的值，則返回Token(ID, result)
        return token
    
    def modules(self):
        result = ''
        # 处理头文件指令的第一个字符，应该是'#'
        if self.current_char == '#':
            result += self.current_char
            self.next()  # 移动到下一个字符

            # 确认当前字符是否为字母或数字
            while self.current_char is not None and self.current_char.isalnum():
                result += self.current_char
                self.next()  # 移动到下一个字符

            if result == '#include':
                # 解析文件名
                file_name = ''
                
                # 确认当前字符是否为'<'
                if self.current_char == '<':
                    self.next()  # 移动到下一个字符

                    # 确认当前字符是否为字母、数字或'.'
                    while self.current_char is not None and (self.current_char.isalnum() or self.current_char == '.'):
                        file_name += self.current_char
                        self.next()  # 移动到下一个字符

                    # 确认当前字符是否为'>'
                    if self.current_char == '>':
                        self.next()  # 移动到下一个字符

                        # 返回三个标记
                        return [Token(TT_HASH, '#'), Token(TT_INCLUDE, 'include'), Token(TT_FILENAME, file_name)]
                
        # 如果未匹配到头文件指令，则继续处理下一个标记
        return self.get_next_token()

RESERVED_KEYWORDS = {
    'INT': Token('INT', 'int'),
    'RETURN': Token('RETURN', 'return')
}

In [1]:
# 讀取測試程式碼
with open('./test/test2.c', 'r') as f:
    code = f.read()

# 初始化詞法分析器
lexer = Lexer(code)
index = 0
# 遞迴地取得下一個標記，直到遇到結束符號
token = lexer.get_next_token()
while token.type != 'EOF':
    print(f'{index}:{token.value}')
    token = lexer.get_next_token()
    index += 1

NameError: name 'Lexer' is not defined